In [ ]:
# -*- coding: UTF-8 -*-

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import pymc3 as pm
import arviz as az

In [ ]:
np.random.seed(123)
trials = 4
theta_real = 0.35 ## unknown value in a real experiment
data = stats.bernoulli.rvs(p=theta_real, size=trials)

In [ ]:
sns.kdeplot(data)

In [ ]:
with pm.Model() as our_first_model:
    Θ = pm.Beta('Θ', alpha=1., beta=1.)
    y = pm.Bernoulli('y', p=Θ, observed=data)
    trace = pm.sample(1000, random_seed=123)

In [ ]:
az.plot_trace(trace)

In [ ]:
summary = az.summary(trace)
summary

In [ ]:
az.plot_posterior(trace)

In [ ]:
az.plot_posterior(trace, rope=(0.45, 0.55))

In [ ]:
az.plot_posterior(trace, ref_val=0.5)

In [ ]:
# Compare two loss functions over the posterior

grid = np.linspace(0, 1, 200)  # Use a grid of 200 for sampling / plotting
Θ_posterior = trace['Θ']

# Loss proportional to absolute value of difference from "actual" mean
loss_f_abs = [np.mean(abs(i - Θ_posterior)) for i in grid]
# Loss proportional to square of difference from "actual" mean
loss_f_quad = [np.mean((i - Θ_posterior) **2) for i in grid]

# Plot the mean loss over all "actual" means using colors specific to each loss function
for loss_f, color in zip([loss_f_abs, loss_f_quad], ['C0', 'C1']):
    minimum = np.argmin(loss_f)
    plt.plot(grid, loss_f, color)  # plot the loss curve over all "actual" means
    plt.plot(grid[minimum], loss_f[minimum], 'o', color=color)  # plot the minimum loss point
    # label the point with the loss value
    plt.annotate(f'{grid[minimum]:.2f}:', (grid[minimum], loss_f[minimum] + 0.03), color=color)
    plt.yticks([])  # No y-ticks
    plt.xlabel(r'$\hat \theta$')
    
    # Not that the minimum of the quadratic loss function occurs at the mean of the posterior 
    # and the minimum of the absolute loss function occurs at the median. See Jaynes, section 13.9, 
    # for a detailed example.

In [ ]:
# A silly example of a complex loss function

losses = []
for i in grid:
    if i < 0.5:
        f = np.mean(np.pi * Θ_posterior / np.abs(i - Θ_posterior))
    else:
        f = np.mean(1 / (i - Θ_posterior))
    losses.append(f)
    
minimum = np.argmin(losses)
plt.plot(grid, losses)
plt.plot(grid[minimum], losses[minimum], 'o')
plt.annotate(f'{grid[minimum]:.2f}', (grid[minimum] + 0.01, losses[minimum] + 0.1))
plt.yticks([])
plt.xlabel(r'$\hat \theta$')

In [ ]:
data = np.loadtxt('./data/chemical_shifts.csv')
az.plot_kde(data, rug=True)
plt.yticks([0], alpha=0)

In [ ]:
with pm.Model() as model_g:
    µ = pm.Uniform('µ', lower=40, upper=70)
    σ = pm.HalfNormal('σ', sd=10)
    y = pm.Normal('y', mu=µ, sd=σ, observed=data)
    trace_g = pm.sample(1000)

In [ ]:
az.plot_trace(trace_g)

In [ ]:
az.plot_joint(trace_g, kind='kde', fill_last=False)

In [ ]:
trace_g['σ']

In [ ]:
az.plot_kde(trace_g['σ'])

In [ ]:
az.summary(trace_g)

In [ ]:
y_pred_g = pm.sample_posterior_predictive(trace_g, 100, model_g)
data_ppc = az.from_pymc3(trace=trace_g, posterior_predictive=y_pred_g)
ax = az.plot_ppc(data_ppc, figsize=(12, 6), mean=False)
ax[0].legend(fontsize=15)


In [ ]:
stats.describe(data)

In [ ]:
# Identify outliers less than Q1 - 1.5 * IQR (Tukey's fences)
data[data < np.percentile(data, 25, interpolation='midpoint') - 1.5 * stats.iqr(data)]

In [ ]:
# Identify outliers greater than Q3 + 1.5 * IQR (Tukey's fences)
data[data > np.percentile(data, 75, interpolation='midpoint') + 1.5 * stats.iqr(data)]

In [ ]:
# Identify outliers less than mean - 2 * std
data[data < np.mean(data) - 2 * np.std(data)]

In [ ]:
# Identify outliers greater than mean + 2 * std
data[data > np.mean(data) + 2 * np.std(data)]

In [ ]:
# Identify outliers greater than mean + 2.5 * std
data[data > np.mean(data) + 2.5 * np.std(data)]

In [ ]:
# Identify outliers greater than mean + 3 * std
data[data > np.mean(data) + 3 * np.std(data)]

In [ ]:
lorentz_means = [np.mean(stats.t(loc=0, df=1).rvs(100)) for _ in range(250)]
# Freedman-Diaconis rule (https://stats.stackexchange.com/questions/798/calculating-optimal-number-of-bins-in-a-histogram)
h = int(np.ceil(2 * stats.iqr(lorentz_means) * (len(lorentz_means) ** (1 / 3))))
t_large_scale_means = [np.mean(stats.t(loc=0, df=100).rvs(100)) for _ in range(250)]
bins = np.linspace(-2, 2, h)
plt.hist([lorentz_means, t_large_scale_means], bins, alpha=0.5, label=['Lorentz Means', 'Large Scale Means'])
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6))
x_values = np.linspace(-10, 10, 500)
for df in [1, 2, 30]:
    distribution = stats.t(df)
    x_pdf = distribution.pdf(x_values)
    plt.plot(x_values, x_pdf, label=fr'$\nu = {df}$', lw=3)

xn_pdf = stats.norm.pdf(x_values)
plt.plot(x_values, xn_pdf, 'k--', label=r'$\nu = \infty$')
plt.xlabel('x')
plt.yticks([])
plt.legend()
plt.xlim(-5, 5)

In [ ]:
with pm.Model() as model_t:
    µ = pm.Uniform('µ', 40, 75)
    σ = pm.HalfNormal('σ', sd=10)
    nu = pm.Exponential('\u03bd', 1/30)
    y = pm.StudentT('y', mu=µ, sd=σ, nu=nu, observed=data)
    trace_t = pm.sample(1000)
az.plot_trace(trace_t)

In [ ]:
az.summary(trace_t)

In [ ]:
y_ppc_t = pm.sample_posterior_predictive(trace_t, 100, model_t, random_seed=123)
y_pred_t = az.from_pymc3(trace=trace_t, posterior_predictive=y_ppc_t)
az.plot_ppc(y_pred_t, figsize=(12, 6), mean=False)
ax[0].legend(fontsize=15)
plt.xlim(40, 70)